In [1]:
import re
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

# Load main data set

In [2]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)
df.head()

,source,target
0,O=C1CC[C@H](CN2CCN(CCOc3cc4ncnc(Nc5ccc(F)c(Cl)...,CS(=O)(=O)OC[C@H]1CCC(=O)O1.Fc1ccc(Nc2ncnc3cc...
1,Nc1nc2[nH]c(CCCc3csc(C(=O)O)c3)cc2c(=O)[nH]1,COC(=O)c1cc(CCCc2cc3c(=O)[nH]c(N)nc3[nH]2)cs1
2,CC1(C)OB(c2cccc(Nc3nccc(C(F)(F)F)n3)c2)OC1(C)C,CC1(C)OB(B2OC(C)(C)C(C)(C)O2)OC1(C)C.FC(F)(F)...
3,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)O,CC(C)(C)OC(=O)NCC(=O)CCC(=O)OCCCC(=O)OCc1ccccc1
4,Fc1cc2c(NC3CCCCCC3)ncnc2cn1,Fc1cc2c(Cl)ncnc2cn1.NC1CCCCCC1


# SMILES Vocabulary: How is it generated?

The model's Vocabulary handles the transformation of SMILES strings into a sequence of tokens. Tokens are the pre-defined lowest and indivisible unit of string text. In Natural Language Processing (NLP), tokens are typically defined on the word or character level. The level of tokenization dictates *what* the model can output, e.g., if tokenization on the character level is used, then the model outputs individual characters.

For generative SMILES models, tokenization is performed on the character level where each token *loosely* maps to a unique atom type (brackets, "(" for example indicate branching and thus, do not map to an atom but rather gives connectivity information).


In [3]:
import sys
sys.path.append('src/')
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

tk = SMILESTokenizer()
vocab = Vocabulary()

#### Tokenization example

In [4]:
# smi_sample = df['source'].iloc[123]
smi_sample = 'CCBr'
print(tk.tokenize(smi_sample, with_begin_and_end=False))

['C', 'C', 'Br']


Much like in a natural language like english where one's vocabulary controls what sentences can be formulated, a molecular generative model's vocabulary controls what kind of atoms can be proposed - sentences in this context are molecules

In order for the token representations of SMILES sequences to be passed into a machine learning model, they must be transformed into a numerical representation. This is done in the Vocabulary class where each unique token is mapped to a numerical index. This is shown below:


In [5]:
# create a vocabulary using all SMILES in df
smiles_dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
smiles_dataset = np.unique(smiles_dataset).tolist()

vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=tk)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'The unique tokens are: \n{vocabulary.tokens()}')

There are 86 unique tokens in the vocabulary.

The unique tokens are: 
['$', '^', ' ', '#', '(', ')', '-', '.', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Br', 'C', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', '[B-]', '[BH-]', '[BH3-]', '[Br-]', '[C-]', '[C@@H]', '[C@@]', '[C@H]', '[C@]', '[Cl+3]', '[Cl-]', '[Cu]', '[Fe]', '[I+]', '[K]', '[Li]', '[Mg+]', '[Mg]', '[N+]', '[N-]', '[N@+]', '[NH+]', '[NH-]', '[NH2+]', '[NH3+]', '[NH4+]', '[O-]', '[OH-]', '[P+]', '[PH2]', '[PH4]', '[PH]', '[Pd]', '[Pt]', '[S+]', '[S-]', '[S@@]', '[S@]', '[SH]', '[Se]', '[SiH2]', '[SiH]', '[Si]', '[SnH]', '[Sn]', '[Zn+]', '[Zn]', '[n+]', '[n-]', '[nH]', '[s+]', '[se]', '\\', 'c', 'n', 'o', 's']


In [17]:
print(smi_sample)
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)

vocabulary.encode(tokenized_smi_sample)

CCBr
['C', 'C', 'Br']


array([21., 21., 20.], dtype=float32)

Τhe `encode` method in the `Vocabulary` class takes a list of tokens and returns its numerical indices the indices are what we expect

# RNN section

This section describes *how* the numerical representation of tokens are transformed into an input vector known as the embedding that will act as the input to the RNN.

An Embedding Layer is essentially a look-up table. In the constructor above, `num_embeddings` refers to the Vocabulary size. 

`num_embeddings` denotes how many vectors to initialize. 

Since we have n unique tokens, we need _ different vectors: 1 for each unique token. This is why `num_embeddings` is _ in this example. `embedding_dim` denotes the dimension of the embedding vector. 5 is arbitrarily chosen here just for easy visualization.



In [33]:
# construct an "Embedding layer"
EMBEDDING_DIM = 5
NUM_EMBEDDING = len(vocabulary)

embedding_layer = nn.Embedding(num_embeddings=NUM_EMBEDDING,
                               embedding_dim=EMBEDDING_DIM)

# only 1 layer of LSTM cells is initialized here for the sake of illustration
# input_size = 5 because we previously defined the "embedding_dim" of the Embedding layer to be 5
# hidden_size = 5 is arbitrarily chosen for easy visualization
recurrent_layer = nn.LSTM(input_size=EMBEDDING_DIM,
                          hidden_size=5,
                          num_layers=1,
                          dropout=0,
                          batch_first=True)

In [34]:
# get the numerical indices of bromoethane
numerical_indices_smi_sample = torch.LongTensor([vocabulary.encode(tokenized_smi_sample).astype(int)])
print(f"Numerical indices of bromoethane:\n {numerical_indices_smi_sample}\n")

embedding = embedding_layer(numerical_indices_smi_sample)
print(f"Embedding:\n {embedding}")

Numerical indices of bromoethane:
 tensor([[21, 21, 20]])

Embedding:
 tensor([[[ 0.1534, -0.3632, -0.1740,  1.8087, -0.1279],
         [ 0.1534, -0.3632, -0.1740,  1.8087, -0.1279],
         [ 1.6083, -0.8310, -1.3849, -0.7949,  0.2275]]],
       grad_fn=<EmbeddingBackward0>)


In [48]:
print(embedding.shape)

# let's run the embedding through the recurrent layer
rnn_output, (hidden_state, cell_state) = recurrent_layer(embedding)

print(rnn_output.shape)



# initialize the linear layer
# in_features = 5 as that is the hidden_size defined in the recurrent layer above
# out_features = 20 as that is the size of the Vocabulary
linear_layer = nn.Linear(in_features=5,
                         out_features=20)

linear_output = linear_layer(rnn_output)

# verify the shape of the linear output is what we expect:
# (batch size) x (sequence length) x (vocabulary size)
print(linear_output.shape)

# let's first show the normal softmax output
# recall the output from the linear layer has dimensions: (batch size) x (sequence length) x (vocabulary size)
# therefore, dim=2 because we want to compute the softmax over the vocabulary to obtain a probability for each token
softmax = linear_output.softmax(dim=2)
print(softmax)

# let's now show the log-softmax output
log_softmax = linear_output.log_softmax(dim=2)
print(log_softmax)

# log-softmax to token probabilities
# recall our original SMILES 
print(f"Original SMILES string: {smi_sample}\n")

# recall our vocabulary
print(f"The unique tokens are: \n{vocabulary.tokens()}\n")

# we now extract the max value in each tensor of the log-softmax output above and the corresponding token
most_probable_tokens = log_softmax.argmax(dim=2).flatten().tolist()
for idx, (correct_token, most_probable_token) in enumerate(zip(smi_sample, most_probable_tokens)):
    print(f"At time step {idx+1}, the generative model proposes {vocabulary.tokens()[most_probable_token]} as the most probable token and the correct token is {correct_token}")

torch.Size([1, 3, 5])
torch.Size([1, 3, 5])
torch.Size([1, 3, 20])
tensor([[[0.0742, 0.0401, 0.0410, 0.0523, 0.0651, 0.0588, 0.0472, 0.0356,
          0.0788, 0.0493, 0.0742, 0.0334, 0.0660, 0.0341, 0.0478, 0.0344,
          0.0509, 0.0434, 0.0397, 0.0338],
         [0.0770, 0.0397, 0.0425, 0.0506, 0.0670, 0.0565, 0.0466, 0.0366,
          0.0791, 0.0493, 0.0738, 0.0338, 0.0663, 0.0342, 0.0464, 0.0338,
          0.0491, 0.0440, 0.0403, 0.0332],
         [0.0665, 0.0439, 0.0417, 0.0549, 0.0586, 0.0675, 0.0510, 0.0323,
          0.0727, 0.0486, 0.0706, 0.0344, 0.0655, 0.0338, 0.0527, 0.0350,
          0.0509, 0.0455, 0.0398, 0.0340]]], grad_fn=<SoftmaxBackward0>)
tensor([[[-2.6014, -3.2175, -3.1939, -2.9500, -2.7316, -2.8341, -3.0533,
          -3.3361, -2.5405, -3.0096, -2.6011, -3.3979, -2.7184, -3.3784,
          -3.0409, -3.3710, -2.9770, -3.1381, -3.2272, -3.3879],
         [-2.5640, -3.2260, -3.1572, -2.9831, -2.7025, -2.8742, -3.0666,
          -3.3071, -2.5376, -3.0095, -2.6061, 

# Train / validation / test split

In [9]:
from sklearn.model_selection import train_test_split

print(df.shape)

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

# Printing the sizes of the resulting splits
print("Train data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

(45033, 2)
Train data size: 36026
Validation data size: 7205
Test data size: 1802


# Build the NMT 

In [10]:
import sys
import pandas as pd
sys.path.append('src/')
import argparse
from pathlib import Path
from smiles_lstm.model.smiles_lstm import SmilesLSTM
from smiles_lstm.model.smiles_trainer import SmilesTrainer
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, create_vocabulary
from smiles_lstm.utils import load
from smiles_lstm.utils.misc import suppress_warnings
from torch.nn.utils.rnn import pad_sequence
import string

In [11]:
train     = train_data.copy()
test      = test_data.copy()
valid     = val_data.copy()

# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

tokenizer = SMILESTokenizer()
vocab     = create_vocabulary(smiles_list=dataset,
                                    tokenizer=tokenizer,
                                    canonical=False)

MAX_LENGTH = max(len(v) for v in dataset)

print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'Max length: {MAX_LENGTH}.\n')
# print(f'The unique tokens are: \n{vocabulary.tokens()}')

There are 86 unique tokens in the vocabulary.

Max length: 198.



In [12]:
tk = SMILESTokenizer()
vocab = Vocabulary()

smi_sample = 'CCBr'
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)
vocabulary.encode(tokenized_smi_sample)

['C', 'C', 'Br']


array([21., 21., 20.], dtype=float32)

### Function for pad sequencing

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

In [14]:
for d in [train, test, valid]:
    for c in d.columns:
        d[c] = d[c].apply(lambda x: tk.tokenize(x, with_begin_and_end=False))
        d[c] = d[c].apply(lambda x: vocabulary.encode(x).astype(int))
        d[c] = d[c].apply(lambda x: pad_sequence(x, MAX_LENGTH))

# Model Building

In [15]:
# Convert the source and target columns into numpy arrays
trainX = np.array(train['source'].tolist())
trainY = np.array(train['target'].tolist())

print(trainX.shape, trainY.shape)

(36026, 198) (36026, 198)


In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

# Define the input shape
input_shape = (trainX.shape[1], 1)  # Assuming you want to feed one feature at a time

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=NUM_EMBEDDING, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH))
model.add(LSTM(units=128, input_shape=input_shape))
model.add(Dense(units=trainY.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainX, trainY, epochs=3, batch_size=2048)

Epoch 1/3
18/18 [==============================] - 3s 26ms/step - loss: -3.1878 - accuracy: 0.0028
Epoch 2/3
18/18 [==============================] - 0s 27ms/step - loss: -23.7747 - accuracy: 0.0012
Epoch 3/3
18/18 [==============================] - 1s 28ms/step - loss: -46.9983 - accuracy: 0.0012


In [17]:
# Convert the source and target columns into numpy arrays
testX = np.array(test['source'].tolist())
testY = np.array(test['target'].tolist())

print(testX.shape, testY.shape)

(1802, 198) (1802, 198)


In [18]:
pred = model.predict(testX)
pd.DataFrame(testX)

,0,1,2,3,4,5,6,7,8,9,...,188,189,190,191,192,193,194,195,196,197
0,21,21,4,21,5,25,4,21,4,18,...,0,0,0,0,0,0,0,0,0,0
1,26,18,21,9,25,21,4,18,26,5,...,0,0,0,0,0,0,0,0,0,0
2,21,82,9,82,4,21,21,25,10,21,...,0,0,0,0,0,0,0,0,0,0
3,21,25,9,21,21,21,4,25,10,21,...,0,0,0,0,0,0,0,0,0,0
4,21,21,4,21,5,4,21,5,26,21,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,26,18,21,4,25,82,9,82,82,83,...,0,0,0,0,0,0,0,0,0,0
1798,21,21,4,21,5,4,21,5,26,21,...,0,0,0,0,0,0,0,0,0,0
1799,21,21,4,18,26,5,21,4,26,5,...,0,0,0,0,0,0,0,0,0,0
1800,21,21,21,21,4,18,26,5,25,82,...,0,0,0,0,0,0,0,0,0,0


In [20]:
pd.DataFrame(testY)

,0,1,2,3,4,5,6,7,8,9,...,188,189,190,191,192,193,194,195,196,197
0,2,21,21,4,21,5,25,4,21,4,...,0,0,0,0,0,0,0,0,0,0
1,2,25,21,36,4,26,5,82,9,82,...,0,0,0,0,0,0,0,0,0,0
2,2,21,82,9,78,82,10,82,82,82,...,0,0,0,0,0,0,0,0,0,0
3,2,21,25,9,21,21,21,4,25,10,...,0,0,0,0,0,0,0,0,0,0
4,2,21,21,4,21,5,4,21,5,26,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,2,25,82,9,82,82,83,82,82,9,...,0,0,0,0,0,0,0,0,0,0
1798,2,21,21,4,21,5,4,21,5,26,...,0,0,0,0,0,0,0,0,0,0
1799,2,21,21,26,21,4,26,21,21,5,...,0,0,0,0,0,0,0,0,0,0
1800,2,21,21,21,21,4,18,26,5,22,...,0,0,0,0,0,0,0,0,0,0
